In [28]:
import copy
from enum import Enum
DEBUG=False
class NFA_Enum(Enum):
    START=0
    TERMINAL=1
    EPSILON=2
    FAILURE=3
    INTERNAL=4
class NFA:
    START=NFA_Enum.START
    TERMINAL=NFA_Enum.TERMINAL
    EPSILON=NFA_Enum.EPSILON
    FAILURE=NFA_Enum.FAILURE
    INTERNAL=NFA_Enum.INTERNAL
    def __init__(self,state=START,transitions=None):
        self.transitions=transitions if transitions is not None else {}
        self.state=state
    def copy(self):
        return NFA(self.state,copy.deepcopy(self.transitions))
    def __call__(self,transition=None):
        if transition in self.transitions:
            if NFA.EPSILON in self.transitions[transition].transitions:
                return self.transitions[transition](NFA.EPSILON)
            return self.transitions[transition]
        if self.state==NFA.TERMINAL:
            return self
        if NFA.EPSILON in self.transitions:
            return self.transitions[NFA.EPSILON](transition)
        return NFA(NFA.FAILURE)
    def add_edge(self,transition,node=None):
        if transition in self.transitions:
                raise ValueError(f"Transition {transition}->{node} already exists")
        if node is None:
            node=NFA(NFA.TERMINAL)
        self.transitions[transition]=node
        if node.state == NFA.TERMINAL:
            node.state=NFA.INTERNAL
        return self
    def add_edges(self,other: 'NFA'):
        for transition,node in other.transitions.items():
            self.add_edge(transition,node)
    def add_self_loop(self, transition):
        self.transitions[transition] = self
        return self
    def __add__(self,other):
        #NFA(START,{X:NFA(INTERNAL,{Y:NFA(TERMINAL)})}
        return self.copy().add_transition(NFA.EPSILON,other.copy())
    def __repr__(self) -> str:
        return f"{self.state=},{self.transitions=}"

def parse_expr(expr):#converts expr to a nfa returns the start and end nodes
    #| for OR
    #? for optional
    #() for grouping
    # for concatenation
    #equal precedence for OR and concatenation
    #words for units - regex [a-zA-Z]+
    #DOES NOT WORK for expressions like (x y)|(x? z) instead do (x y|z)|z
    
    stack=[NFA(NFA.START)]
    operator = ' '
    i=0
    az="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    last_optional=False
    while i<len(expr):
        sub_start,sub_end=None,None
        match expr[i]:    
            case '(':#could optimize by saving location of internal pairs
                open_paren=1
                sub_expr_len=0
                while open_paren>0:
                    i+=1
                    sub_expr_len+=1
                    if expr[i]=='(':
                        open_paren+=1
                    elif expr[i]==')':
                        open_paren-=1
                sub_start,sub_end=parse_expr(expr[i-sub_expr_len+1:i])         
            case s if s in " |":
                operator=s
            case '?':
                last_optional=True
            case s if s in az:
                start=i
                while i<len(expr) and expr[i] in az:
                    i+=1
                sub_end=NFA(NFA.INTERNAL)
                sub_start=NFA(NFA.INTERNAL,transitions={expr[start:i]:sub_end})
                i-=1
        if sub_start is not None:
            if operator == ' ':
                stack[-1].add_edges(sub_start)#instead of an epsilon transition
                stack.append(sub_end)
                if last_optional:
                    stack[-3].add_edges(sub_start)
                    last_optional=False
            elif operator == '|':
                stack[-2].add_edges(sub_start)
                sub_end.add_edge(NFA.EPSILON,stack[-1])
                #treat x?|y as (x|y)?
                      
        i+=1
    stack[-1].state=NFA.TERMINAL
    if last_optional:
        stack[-2].state=NFA.TERMINAL
    return stack[0],stack[-1]


In [35]:
import pyautogui as pag
from time import sleep
from string import ascii_uppercase
"""shift quote comma shift quote period J O I N shift nine left bracket F shift
quote shift backslash shift left bracket I shift plus shift right bracket shift backslash shift quote"""
function_numbers = {
    'one':'1',
    'two':'2',
    'three':'3',
    'four':'4',
    'five':'5',
    'six':'6',
    'seven':'7',
    'eight':'8',
    'nine':'9',
    'ten':'10',
    'eleven':'11',
    'twelve':'12',
}
# Define patterns for each character
patterns = {
  '`': 'backtick|(grave accent)',
 '[': 'opening (square)? bracket',
 ']': 'closing (square)? bracket',
 '\\': 'backslash',
 ';': 'semicolon',
 "'": '(single? quote)|apostrophe',
 ',': 'comma',
 ' ': 'space',
 '.': 'period|dot',
 '/': 'forward slash',
 '=': 'equals|equal sign',
 '-': 'dash|minus|hyphen',
 }|{
   '~':'tilde',
    '!':'exclamation mark',
    '@':'at sign',
    '#':'hashtag|pound sign',
    '$':'dollar sign',
    '%':'percent sign',
    '^':'caret',
    '&':'ampersand',
    '*':'asterisk',
    '(':'opening parenthesis',
    ')':'closing parenthesis',
    '_':'underscore',
    '+':'plus',
    '{':'opening (curly bracket)|brace)',
    '}':'closing (curly bracket)|brace)',
    '|':'pipe|(vertical bar)',
    ':':'colon',
    '"': 'double quote',
     '<': 'opening angle bracket',
   '>': 'closing angle bracket',
     '<': 'opening angle bracket',
 '>': 'closing angle bracket',
 }|{
   'shift':'shift',
    'ctrl':'control',
    'alt':'alt',
 }
pattern_nfas = {
    key:parse_expr(pattern)[0] for key,pattern in patterns.items()
}

state_commands = ['hold','release','function','repeat']
holds = ['shift','ctrl','alt' ]
uppers=list(ascii_uppercase)
other_string = ['enter','escape','backspace','delete','del','up','down','left','right']
def convert(unit,state={"nfas":pattern_nfas}):
  #patterns try to match first
  next_nfas={}
  for key,nfa in state["nfas"].items():
    x=nfa(unit)
    if  x.state == NFA.TERMINAL:
      #reset nfas
      state["nfas"]=pattern_nfas
      return key
    elif x.state != NFA.FAILURE:
      next_nfas[key]=nfa(unit)
    
  state["nfas"]=next_nfas if next_nfas else pattern_nfas#reset when all have failed
  if unit in uppers+state_commands+other_string:#pyautogui supports these other keyboard inputs directly
    return unit.lower()
  if unit in function_numbers:
    return function_numbers[unit]
  return None#invalid unit or patterns incomplete
    
def output_transcription_unit(unit,state={'hold':False,'release':False,'function':False,'repeat':False,'last_pressed':None,'queued':[]}):
  if DEBUG:
    print('output_transcription_unit',unit,state)
  unit=convert(unit)
  if unit is None:
    return None
  elif unit in state_commands:
    state[unit]=True
  elif state['hold']:
    pag.keyDown(unit)
    print("key down",unit)  
    state['release']=False
    state['hold']=False
    if unit in state:
        state[unit]=True
  elif state['release']:
    pag.keyUp(unit)
    print("key up",unit)  
    state['release']=False
    state['hold']=False
    if unit in state['queued']:
        queued=[x for x in queued if x!=unit]
    if unit in state:
        state[unit]=False
  elif unit in holds:
      state['queued'].append(unit)
  elif unit in function_numbers.values() and state['function']:
      state['function']=False
      output_transcription_unit('f'+unit)
  elif state['repeat']:
    ... 
  else:
    state['last_pressed']=[*state['queued'],unit]
    pag.hotkey(*state['last_pressed'])
    print("press", "+".join(state['last_pressed']))
    state['queued']=[] 
DEBUG=True
sample = "control shift S"#"""P R I N T opening parenthesis shift quote shift H E L L O shift W O R L D left left left left left space"""
for word in sample.split(): 
  output_transcription_unit(word)

output_transcription_unit control {'hold': False, 'release': False, 'function': False, 'repeat': False, 'last_pressed': None, 'queued': []}
output_transcription_unit shift {'hold': False, 'release': False, 'function': False, 'repeat': False, 'last_pressed': None, 'queued': ['ctrl']}
output_transcription_unit S {'hold': False, 'release': False, 'function': False, 'repeat': False, 'last_pressed': None, 'queued': ['ctrl', 'shift']}
press ctrl+shift+s
